In [4]:
from gurobipy import *
import numpy as np
import pandas as pd
import math
import time
from datetime import datetime, timedelta, date
import holidays
from calendar import isleap

#### Creating Model

In [5]:
m = Model('clothing manufacturing model')

#### Setting start date

In [6]:
# get days in each month
def get_day_dict(year):
    # Base dictionary for days in each month
    day_dict = {1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30,
                7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31}
    # Adjust February for leap years
    if isleap(year):
        day_dict[2] = 29
    return day_dict

In [7]:
year = 2024
month = 8
day = 1
start_date = datetime(year, month, day) 

In [8]:
# holidays in Taiwan
tw_holidays = holidays.TW(years=year)
print(tw_holidays)

{datetime.date(2024, 1, 1): '中華民國開國紀念日', datetime.date(2024, 2, 9): '農曆除夕', datetime.date(2024, 2, 10): '春節', datetime.date(2024, 2, 11): '春節', datetime.date(2024, 2, 12): '春節', datetime.date(2024, 2, 28): '和平紀念日', datetime.date(2024, 4, 4): '兒童節; 民族掃墓節', datetime.date(2024, 6, 10): '端午節', datetime.date(2024, 9, 17): '中秋節', datetime.date(2024, 10, 10): '國慶日', datetime.date(2024, 4, 5): '兒童節（補假）', datetime.date(2024, 2, 13): '春節（補假）', datetime.date(2024, 2, 14): '春節（補假）', datetime.date(2024, 2, 8): '休息日（2024-02-17日起取代）'}


#### Setting Order Number

In [9]:
# 8, 10, 12, 14, 16
order_num = 16

#### Creating Set

In [31]:
# --- Calendar & working-day index ---
day_dict = get_day_dict(year) # {month: days_in_month}; Feb handles leap years internally
Num_L = day_dict[month] # number of calendar days in target month

# Set of calendar day indices (0-based) that are non-working (weekends or holidays)
# Example: if Aug 3 is Saturday, '2' will be in this set.
HL = set()
for day in range(1, day_dict[month] + 1):
    current_date = date(year, month, day)
    if current_date.weekday() >= 5 or current_date in tw_holidays:  # Check for weekends or holidays
        HL.add(day - 1) # day index is start from zero

# Map working-day index -> calendar day-of-month (both 0-based)
# Example: if Aug 1 (0) is working, L_date[0] == 0; if Aug 2 is holiday, it is skipped.
L_date = dict()
no = 0 
for l in range(Num_L):
    if l not in HL:
        L_date[no] = l
        no += 1

# Working-day index set (e.g., {0,1,2,...} counting only working days)
L = set(L_date.keys())

# --- Orders ---
Num_K = order_num # number of orders
K = set(range(Num_K)) # order index set {0..Num_K-1}

# --- Equipment ---
# Number of machines for machine type
Ed = [10, 10]
# Total number of machines
Num_E = sum(Ed)
# Machine index set: {0, 1, ..., Num_E-1}
E = set(range(Num_E))

# Number of bearings (Machine type 1: 2, Machine type 2: 3)
Pd = [2, 3]
# Total number of bearings 
Num_P = int(np.dot(Ed, Pd))
# Bearing index set: {0, 1, ..., Num_P-1}
P = set(range(Num_P))

# --- Order sides (e.g., A/B) ---
# Number of sides per order (2 → sides A and B)
Num_I = 2
# Side index set: {0, 1}  (map 0 to 'A', 1 to 'B')
I = set(range(Num_I))
# Number of stages per order (2 → stages 1 and 2)
Num_J = 2

#### Creating Parameter

In [32]:
# Reading Order Data
# order quantity
with open(f"./data/order/{order_num}/quantity.txt", 'r') as file:
    # Read the content of the file and split it 
    content = file.read()
    q = list(map(int, content.split()))

# order value
with open(f"./data/order/{order_num}/value.txt", 'r') as file:
    # Read the content of the file and split it 
    content = file.read()
    v = list(map(int, content.split()))

# order due date
with open(f"./data/order/{order_num}/due_date.txt", 'r') as file:
    # Read the content of the file and split it 
    content = file.read()
    u = list(map(int, content.split()))
    u = [d - 1 for d in u]  # convert from 1-based day numbers to 0-based indices    

In [33]:
# --- Penalty & processing parameters ---
r = 0.01 # tardiness penalty weight
a = 8  # stage 1 processing duration
b = 100  # stage 2 throughput capacity
M = 9999

#### Creating Variable

In [34]:
x = m.addVars(Num_L, Num_K, Num_I, Num_P, Num_J - 1, vtype=GRB.BINARY, name="x")
y = m.addVars(Num_L, Num_K, Num_I, Num_P, Num_J, vtype=GRB.BINARY, name="y")
t = m.addVars(Num_K, Num_I, vtype=GRB.INTEGER, name="t")
w = m.addVars(Num_K, Num_I, vtype=GRB.INTEGER, name="w")
c = m.addVars(Num_K, vtype=GRB.INTEGER, name="c")

#### Update

In [35]:
m.update()

#### Objective Function

In [36]:
# summation vk * r * ck
sum_vrc = LinExpr()
for k in K:
    sum_vrc += v[k] * r * c[k]
    
# setting obj.
m.setObjective(sum_vrc, GRB.MINIMIZE)

#### Constraints

In [18]:
# constraint 1
for l in L:
    for p in P:
        c1_sum_x = LinExpr()
        for k in K:
            for i in I:
                c1_sum_x += x[l, k, i, p, 0]
    
        m.addConstr(c1_sum_x <= 1, "c1")

In [19]:
# constraint 2
for l in L:
    for p in P:
        c2_sum_y = LinExpr()
        for k in K:
            for i in I:
                c2_sum_y += y[l, k, i, p, 0] 
                c2_sum_y += y[l, k, i, p, 1] 

        m.addConstr(c2_sum_y <= 1, "c2")

In [20]:
# constraint 3
for k in K:
    for i in I:
        c3_sum_x = LinExpr()
        for l in L:
            for p in P:
                c3_sum_x += x[l, k, i, p, 0]

        m.addConstr(c3_sum_x == 1, "c3")

In [21]:
# constraint 4
for k in K:
    for i in I:
        for p in P:
            for l in range(Num_L - a + 1):
                c4_sum_y = LinExpr()
                for l_p in range(l, l + a):
                    c4_sum_y += y[l_p, k, i, p, 0]

                m.addConstr(c4_sum_y >= - M * (1 - x[l, k, i, p, 0]) + a, "c4")
                m.addConstr(c4_sum_y <= M * (1 - x[l, k, i, p, 0]) + a, "c4")
                                            

In [22]:
# constraint 5
for k in K:
    for i in I:
        for p in P:
            c5_sum_x = LinExpr()
            c5_sum_y = LinExpr()
            for l in L:
                c5_sum_x += x[l, k, i, p, 0]
                c5_sum_y += y[l, k, i, p, 0]

            m.addConstr(c5_sum_x * a >= c5_sum_y, "c5")
                                             

In [23]:
# constraint 6
for k in K:
    for i in I:
        for l_p in L:
            for p_p in P:
                c8_sum_x = LinExpr()
                for l in L:
                    for p in P:
                        c8_sum_x += x[l, k, i, p, 0] * (l + a)

                m.addConstr(
                    M * (1 -  y[l_p, k, i, p_p, 1]) + l_p * y[l_p, k, i, p_p, 1] >= c8_sum_x, "c6")
                               

In [24]:
# constraint 7
for k in K:
    for i in I:
        c7_sum_y = LinExpr()
        for l in L:
            for p in P:
                c7_sum_y += y[l, k, i, p, 1]

        m.addConstr(c7_sum_y == math.ceil(q[k] / b), "c7")
                                       
                    

In [25]:
# constraint 8
for l in L:
    c8_sum_y = LinExpr()
    for k in K:
        for i in I:
            for p in P:
                c8_sum_y += y[l, k, i, p, 0]
                c8_sum_y += y[l, k, i, p, 1]

    m.addConstr(c8_sum_y <= 50, "c8")

In [26]:
# constraint 9
for l in L:
    for k in K:
        for i in I:
            for p in P:
                # using date
                m.addConstr(L_date[l] * y[l, k, i, p, 1] - u[k] <= t[k, i], "c9")

In [27]:
# constraint 10 
for k in K:
    for i in I:
        # gurobi doesn't support strict inequality, so we add 1e-5 in first constraint
        m.addConstr(7 * (w[k, i] - 1) + 1e-5 <=  t[k, i], "c10_l")
        m.addConstr(7 * w[k, i] >= t[k, i], "c10_r")

In [28]:
# constraint 11
for k in K:
    for i in I:
        m.addConstr(w[k, i] <= c[k], "c11")

#### Result

In [29]:
# record cpu time
start_time = time.time()

# set time limit 600 sec
m.setParam('TimeLimit', 600)
m.optimize()

# End the timer 
cpu_time = time.time() - start_time

Set parameter TimeLimit to value 600
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Non-default parameters:
TimeLimit  600

Optimize a model with 151182 rows, 148880 columns and 39832192 nonzeros
Model fingerprint: 0x20a0d6d6
Variable types: 0 continuous, 148880 integer (148800 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 65300 rows and 43200 columns (presolve time = 6s)...
Presolve removed 65332 rows and 43232 columns (presolve time = 10s)...
Presolve removed 65386 rows and 43232 columns (presolve time = 16s)...
Presolve removed 65418 rows and 43264 columns (presolve time = 20s)...
Presolve removed 65418 rows and 43264 columns (presolve time = 25s)...
Presolve removed 65768 rows and 43264 columns (presolve t

In [30]:
print(f"cpu time: {cpu_time:.2f} seconds")

cpu time: 135.97 seconds


In [28]:
# x results
for k in K:
    for i in I:
        for p in P:       
            for l in L:
                if x[l, k, i, p, 0].x > 0:
                    print(f"Order {k} Side {i} stage 0 is starting to print on Bearing {p} at time {l}")


Order 0 Side 0 stage 0 is starting to print on Bearing 17 at time 0
Order 0 Side 1 stage 0 is starting to print on Bearing 36 at time 0
Order 1 Side 0 stage 0 is starting to print on Bearing 38 at time 0
Order 1 Side 1 stage 0 is starting to print on Bearing 40 at time 0
Order 2 Side 0 stage 0 is starting to print on Bearing 15 at time 0
Order 2 Side 1 stage 0 is starting to print on Bearing 44 at time 2
Order 3 Side 0 stage 0 is starting to print on Bearing 12 at time 0
Order 3 Side 1 stage 0 is starting to print on Bearing 8 at time 0
Order 4 Side 0 stage 0 is starting to print on Bearing 41 at time 0
Order 4 Side 1 stage 0 is starting to print on Bearing 10 at time 0
Order 5 Side 0 stage 0 is starting to print on Bearing 9 at time 0
Order 5 Side 1 stage 0 is starting to print on Bearing 27 at time 1
Order 6 Side 0 stage 0 is starting to print on Bearing 3 at time 0
Order 6 Side 1 stage 0 is starting to print on Bearing 11 at time 0
Order 7 Side 0 stage 0 is starting to print on Bear

In [30]:
# y results
for k in K:
    for i in I:
        for j in [0, 1]:
            for p in P:       
                for l in L:
                    if y[l, k, i, p, j].x > 0:
                        print(f"Order {k} Side {i} stage {j} is printing on Bearing {p} at time {l}")
   

Order 0 Side 0 stage 0 is printing on Bearing 17 at time 0
Order 0 Side 0 stage 0 is printing on Bearing 17 at time 1
Order 0 Side 0 stage 0 is printing on Bearing 17 at time 2
Order 0 Side 0 stage 0 is printing on Bearing 17 at time 3
Order 0 Side 0 stage 0 is printing on Bearing 17 at time 4
Order 0 Side 0 stage 0 is printing on Bearing 17 at time 5
Order 0 Side 0 stage 0 is printing on Bearing 17 at time 6
Order 0 Side 0 stage 0 is printing on Bearing 17 at time 7
Order 0 Side 0 stage 1 is printing on Bearing 0 at time 13
Order 0 Side 0 stage 1 is printing on Bearing 2 at time 10
Order 0 Side 0 stage 1 is printing on Bearing 6 at time 13
Order 0 Side 0 stage 1 is printing on Bearing 10 at time 13
Order 0 Side 0 stage 1 is printing on Bearing 13 at time 13
Order 0 Side 0 stage 1 is printing on Bearing 14 at time 12
Order 0 Side 0 stage 1 is printing on Bearing 14 at time 13
Order 0 Side 0 stage 1 is printing on Bearing 16 at time 13
Order 0 Side 0 stage 1 is printing on Bearing 17 at

In [31]:
# t results
for k in K:
    for i in I:
        if t[k, i].x > 0:
            print(f"Order {k} Side {i} is tardy {t[k, i].x} days")
        

In [32]:
# w results
for k in K:
    for i in I:
        if w[k, i].x > 0:
            print(f"Order {k} Side {i} is tardy {w[k, i].x} weeks")

In [33]:
# k results
for k in K:
    for i in I:
        if c[k].x > 0:
            print(f"Order {k} is tardy {c[k].x} weeks")

In [34]:
# objVal
print('Obj: %g' % m.objVal)

Obj: 0


#### Generating Excel

In [50]:
# Create dataframe 
tuple_list = []
# Two types of machines
p = 0
while p < len(P):
    for d in [0, 1]: 
        for e in range(Ed[d]):
            if d > 0:
                e += sum(Ed[:d])
            for _ in range(Pd[d]):
                tuple_list.append((f"Equipment {e}", f"Bearing {p}" ))
                p+=1
                

col_name = pd.MultiIndex.from_tuples(tuple_list, names=['Equipment', 'Bearing'])
# using all date
df = pd.DataFrame(index=[f"Day {day}" for day in range(Num_L)], columns = col_name)

In [51]:
df

Equipment Equipment 0           Equipment 1           Equipment 2            \
Bearing     Bearing 0 Bearing 1   Bearing 2 Bearing 3   Bearing 4 Bearing 5   
Day 0             NaN       NaN         NaN       NaN         NaN       NaN   
Day 1             NaN       NaN         NaN       NaN         NaN       NaN   
Day 2             NaN       NaN         NaN       NaN         NaN       NaN   
Day 3             NaN       NaN         NaN       NaN         NaN       NaN   
Day 4             NaN       NaN         NaN       NaN         NaN       NaN   
Day 5             NaN       NaN         NaN       NaN         NaN       NaN   
Day 6             NaN       NaN         NaN       NaN         NaN       NaN   
Day 7             NaN       NaN         NaN       NaN         NaN       NaN   
Day 8             NaN       NaN         NaN       NaN         NaN       NaN   
Day 9             NaN       NaN         NaN       NaN         NaN       NaN   
Day 10            NaN       NaN         NaN       NaN         NaN       NaN   
Day 11            NaN       NaN         NaN       NaN         NaN       NaN   
Day 12            NaN       NaN         NaN       NaN         NaN       NaN   
Day 13            NaN       NaN         NaN       NaN         NaN       NaN   
Day 14            NaN       NaN         NaN       NaN         NaN       NaN   
Day 15            NaN       NaN         NaN       NaN         NaN       NaN   
Day 16            NaN       NaN         NaN       NaN         NaN       NaN   
Day 17            NaN       NaN         NaN       NaN         NaN       NaN   
Day 18            NaN       NaN         NaN       NaN         NaN       NaN   
Day 19            NaN       NaN         NaN       NaN         NaN       NaN   
Day 20            NaN       NaN         NaN       NaN         NaN       NaN   
Day 21            NaN       NaN         NaN       NaN         NaN       NaN   
Day 22            NaN       NaN         NaN       NaN         NaN       NaN   
Day 23            NaN       NaN         NaN       NaN         NaN       NaN   
Day 24            NaN       NaN         NaN       NaN         NaN       NaN   
Day 25            NaN       NaN         NaN       NaN         NaN       NaN   
Day 26            NaN       NaN         NaN       NaN         NaN       NaN   
Day 27            NaN       NaN         NaN       NaN         NaN       NaN   
Day 28            NaN       NaN         NaN       NaN         NaN       NaN   
Day 29            NaN       NaN         NaN       NaN         NaN       NaN   
Day 30            NaN       NaN         NaN       NaN         NaN       NaN   

Equipment Equipment 3           Equipment 4            ... Equipment 16  \
Bearing     Bearing 6 Bearing 7   Bearing 8 Bearing 9  ...   Bearing 40   
Day 0             NaN       NaN         NaN       NaN  ...          NaN   
Day 1             NaN       NaN         NaN       NaN  ...          NaN   
Day 2             NaN       NaN         NaN       NaN  ...          NaN   
Day 3             NaN       NaN         NaN       NaN  ...          NaN   
Day 4             NaN       NaN         NaN       NaN  ...          NaN   
Day 5             NaN       NaN         NaN       NaN  ...          NaN   
Day 6             NaN       NaN         NaN       NaN  ...          NaN   
Day 7             NaN       NaN         NaN       NaN  ...          NaN   
Day 8             NaN       NaN         NaN       NaN  ...          NaN   
Day 9             NaN       NaN         NaN       NaN  ...          NaN   
Day 10            NaN       NaN         NaN       NaN  ...          NaN   
Day 11            NaN       NaN         NaN       NaN  ...          NaN   
Day 12            NaN       NaN         NaN       NaN  ...          NaN   
Day 13            NaN       NaN         NaN       NaN  ...          NaN   
Day 14            NaN       NaN         NaN       NaN  ...          NaN   
Day 15            NaN       NaN         NaN       NaN  ...          NaN   
Day 16            NaN       NaN         Na

In [52]:
# Fill Scheduling DataFrame
for k in K:
    for i in I:
        for j in [0, 1]:
            for p in P:     
                for l in L:
                    if y[l, k, i, p, j].x > 0:
                        if i == 0: 
                            side = "A"
                        else:
                            side = "B"
                             
                        col_mask = df.columns.get_level_values(1) == f"Bearing {p}"
                        df.loc[f"Day {L_date[l]}", col_mask] = f"Order {k} Side {side} Stage {j+1}"
                        # print(f"Order {k} Side {i} stage {j} is printing on Equipment {e} Bearing {p} at time {l}")


In [53]:
# Replace day no. with date 
df.index = df.index.map(lambda x: (start_date + timedelta(days=int(x.split()[1]))).strftime('%m/%d'))

In [54]:
# Use color to represent different orders
color_pairs = {
    'Order 0': ('#1f77b4', '#aec7e8'),
    'Order 1': ('#ff7f0e', '#ffbb78'),
    'Order 2': ('#2ca02c', '#98df8a'),
    'Order 3': ('#d62728', '#ff9896'),
    'Order 4': ('#9467bd', '#c5b0d5'),
    'Order 5': ('#8c564b', '#c49c94'),
    'Order 6': ('#e377c2', '#f7b6d2'),
    'Order 7': ('#7f7f7f', '#c7c7c7'),
    'Order 8': ('#bcbd22', '#dbdb8d'),
    'Order 9': ('#17becf', '#9edae5'),
    'Order 10': ('#393b79', '#5254a3'),
    'Order 11': ('#637939', '#8ca252'),
    'Order 12': ('#8c6d31', '#bd9e39'),
    'Order 13': ('#843c39', '#ad494a'),
    'Order 14': ('#7b4173', '#a55194'),
    'Order 15': ('#17becf', '#98df8a'),
}

In [55]:
# Save the Excel
with pd.ExcelWriter(f"./schedule_result/result/{order_num}/scheduling_gurobi_order_{order_num}_practical.xlsx") as writer:
    df.to_excel(writer, sheet_name="Schedule")
    
    # Access the xlsxwriter workbook and worksheet objects
    workbook  = writer.book
    worksheet = writer.sheets["Schedule"]
    
    # Set the width of each column
    for col_num, col in enumerate(df.columns, start=1):
        worksheet.set_column(col_num, col_num, 25)

    # set color for each order 
    for col_num, col in enumerate(df.columns, start=1):
        for row_num, value in enumerate(df[col], start=3):
            if pd.notna(value):
                # split the value, which is Order k Side i Stage j
                split_str = value.split()
                order_key = split_str[0] + " " + split_str[1]

                if "A" in split_str :
                    color = color_pairs[order_key][0]
                else:
                    color = color_pairs[order_key][1]
                          
                worksheet.write(row_num, col_num, value, workbook.add_format({'bg_color': color}))
